# VIBE: Video Inference for Human Body Pose and Shape Estimation

Demo of the original PyTorch based implementation provided here: https://github.com/mkocabas/VIBE

## Note
Before running this notebook make sure that your runtime type is 'Python 3 with GPU acceleration'. Go to Edit > Notebook settings > Hardware Accelerator > Select "GPU".

## More Info
- Paper: https://arxiv.org/abs/1912.05656
- Repo: https://github.com/mkocabas/VIBE

In [0]:
# Clone the repo
!git clone https://github.com/mkocabas/VIBE.git
# Install dependencies
%cd VIBE/
!pip install torch numpy
!pip install -r requirements.txt
# Download pretrained weights and SMPL data
!sh prepare_data.sh

Cloning into 'VIBE'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 115 (delta 48), reused 54 (delta 10), pack-reused 0
Receiving objects: 100% (115/115), 73.52 KiB | 3.50 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/VIBE
  Cloning https://github.com/mattloper/chumpy.git to /tmp/pip-req-build-z1am1o3d
  Running command git clone -q https://github.com/mattloper/chumpy.git /tmp/pip-req-build-z1am1o3d
  Cloning https://github.com/mkocabas/yolov3-pytorch.git to /tmp/pip-req-build-j1f422zb
  Running command git clone -q https://github.com/mkocabas/yolov3-pytorch.git /tmp/pip-req-build-j1f422zb
  Cloning https://github.com/mkocabas/multi-person-tracker.git to /tmp/pip-req-build-5da0owfo
  Running command git clone -q https://github.com/mkocabas/multi-person-tracker.git /tmp/pip-req-build-5da0owfo
     |████████████████████████████████| 399kB 10.4MB/s 
     |█████████████

Downloading...
From: https://drive.google.com/uc?id=1untXhYOLQtpNEy4GTY_0fL_H-k6cTf_r
To: /content/VIBE/data/vibe_data.zip
330MB [00:03, 92.3MB/s]
Archive:  vibe_data.zip
   creating: vibe_data/
  inflating: vibe_data/smpl_mean_params.npz  
  inflating: vibe_data/vibe_model_w_3dpw.pth.tar  
  inflating: vibe_data/vibe_model_wo_3dpw.pth.tar  
  inflating: vibe_data/J_regressor_extra.npy  
  inflating: vibe_data/gmm_08.pkl    
  inflating: vibe_data/SMPL_NEUTRAL.pkl  
  inflating: vibe_data/spin_model_checkpoint.pth.tar  
  inflating: vibe_data/sample_video.mp4  


In [0]:
import os
from os.path import exists, join, basename, splitext

% cd /content
# see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
# clone openpose
!git clone https://github.com/soulslicer/openpose.git
% cd openpose
!git checkout staf
!sed -i 's≈/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
# build openpose
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

/content
Cloning into 'openpose'...
remote: Enumerating objects: 18014, done.
remote: Total 18014 (delta 0), reused 0 (delta 0), pack-reused 18014
Receiving objects: 100% (18014/18014), 79.22 MiB | 36.61 MiB/s, done.
Resolving deltas: 100% (12981/12981), done.
/content/openpose
Branch 'staf' set up to track remote branch 'staf' from 'origin'.
Switched to a new branch 'staf'
sed: can't read openpose/CMakeLists.txt: No such file or directory
Selecting previously unselected package libgflags2.2.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Select

In [0]:
# Get STAF specific models
! cd /content/openpose/models && sh getModels.sh

--2020-01-22 09:39:46--  http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/body_21a_video/pose_iter_264000.caffemodel
Resolving posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)... 128.2.176.37
Connecting to posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)|128.2.176.37|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202598919 (193M) [text/plain]
Saving to: ‘pose/body_21a_video/pose_iter_264000.caffemodel’

pose_iter_264000.ca 100%[===================>] 193.21M  61.5MB/s    in 3.6s    

2020-01-22 09:39:49 (53.5 MB/s) - ‘pose/body_21a_video/pose_iter_264000.caffemodel’ saved [202598919/202598919]

--2020-01-22 09:39:49--  http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/body_25/pose_iter_584000.caffemodel
Resolving posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)... 128.2.176.37
Connecting to posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)|128.2.176.37|:80... connected.
HTTP request sent, await

### Run the demo code.

Check https://github.com/mkocabas/VIBE/blob/master/doc/demo.md for more details about demo.

**Note:** Final rendering is slow compared to inference. We use pyrender with GPU accelaration and it takes 2-3 FPS per image. Please let us know if you know any faster alternative. 

In [0]:
% cd /content/VIBE
# Run the demo
!python demo.py --vid_file dance.mp4 --output_folder output/ --tracking_method pose --staf_dir /content/openpose --run_smplify --sideview --save_obj

# You may use --sideview flag to enable from a different viewpoint, note that this doubles rendering time.
# !python demo.py --vid_file sample_video.mp4 --output_folder output/ --sideview

# You may also run VIBE on a YouTube video by providing a link
# !python demo.py --vid_file https://www.youtube.com/watch?v=BZt_JNBs_TA --output_folder output/ --display

/content/VIBE
Running "ffmpeg -i dance.mp4 -f image2 -v error /tmp/dance_mp4/%06d.png"
Images saved to "/tmp/dance_mp4"
Input video number of frames 242
Executing build/examples/openpose/openpose.bin --model_pose BODY_21A --tracking 1 --render_pose 0 --video /content/VIBE/dance.mp4 --write_json /tmp/dance.mp4_posetrack --display 0
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
RUNNING PoseExtractorCaffeStaf::PoseExtractorCaffeStaf
Empty frame detected, frame number 242 of 256. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():285
Empty frame detected, frame number 242 of 256. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():285
Empty frame detected, frame number 242 of 256. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():285
OpenPose demo successfully finished. Total time: 19.849783 sec

# New Section

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Play the generated video
from IPython.display import HTML
from base64 import b64encode

def video(path):
  mp4 = open(path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML('<video width=500 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

video('output/dance/dance_vibe_result.mp4')